## DSCC202-402 Data Science at Scale Final Project
### Tracking Tweet sentiment at scale using a pretrained transformer (classifier)
<p>Consider the following illustration of the end to end system that you will be building.  Each student should do their own work.  The project will demonstrate your understanding of Spark Streaming, the medalion data architecture using Delta Lake, Spark Inference at Scale using an MLflow packaged model as well as Exploritory Data Analysis and System Tracking and Monitoring.</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/pipeline.drawio.png">

<p>
You will be pulling an updated copy of the course GitHub repositiory: <a href="https://github.com/lpalum/dscc202-402-spring2024">The Repo</a>.  If you are unclear on how to pull an updated copy using the GitHub command line, the following <a href="https://techwritingmatters.com/how-to-update-your-forked-repository-on-github">document</a> is helpful.  Be sure to add the professors and TAs as collaborators on your project. 

- lpalum@gmail.com GitHub ID: lpalum
- ajay.anand@rochester.edu GitHub ID: ajayan12
- divyamunot1999@gmail.com GitHub ID: divyamunot
- ylong6@u.Rochester.edu GitHub ID: NinaLong2077

Once you have updates your fork of the repository you should see the following template project that is resident in the final_project directory.
</p>

<img src="https://data-science-at-scale.s3.amazonaws.com/images/notebooks.drawio.png">

<p>
You can then pull your project into the Databrick Workspace using the <a href="https://www.databricks.training/step-by-step/importing-courseware-from-github/index.html">Repos</a> feature.
Each student is expected to submit the URL of their project on GitHub with their code checked in on the main/master branch.  This illustration highlights the branching scheme that you may use to work on your code in steps and then merge your submission into your master branch before submitting.
</p>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/github.drawio.png">
<p>
Work your way through this notebook which will give you the steps required to submit a complete and compliant project.  The following illustration and associated data dictionary specifies the transformations and data that you are to generate for each step in the medallion pipeline.
</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/dataframes.drawio.png">

#### Bronze Data - raw ingest
- date - string in the source json
- user - string in the source json
- text - tweet string in the source json
- sentiment - the given sentiment of the text as determined by an unknown model that is provided in the source json
- source_file - the path of the source json file the this row of data was read from
- processing_time - a timestamp of when you read this row from the source json

#### Silver Data - Bronze Preprocessing
- timestamp - convert date string in the bronze data to a timestamp
- mention - every @username mentioned in the text string in the bronze data gets a row in this silver data table.
- cleaned_text - the bronze text data with the mentions (@username) removed.
- sentiment - the given sentiment that was associated with the text in the bronze table.

#### Gold Data - Silver Table Inference
- timestamp - the timestamp from the silver data table rows
- mention - the mention from the silver data table rows
- cleaned_text - the cleaned_text from the silver data table rows
- sentiment - the given sentiment from the silver data table rows
- predicted_score - score out of 100 from the Hugging Face Sentiment Transformer
- predicted_sentiment - string representation of the sentiment
- sentiment_id - 0 for negative and 1 for postive associated with the given sentiment
- predicted_sentiment_id - 0 for negative and 1 for positive assocaited with the Hugging Face Sentiment Transformer

#### Application Data - Gold Table Aggregation
- min_timestamp - the oldest timestamp on a given mention (@username)
- max_timestamp - the newest timestamp on a given mention (@username)
- mention - the user (@username) that this row pertains to.
- negative - total negative tweets directed at this mention (@username)
- neutral - total neutral tweets directed at this mention (@username)
- positive - total positive tweets directed at this mention (@username)

When you are designing your approach, one of the main decisions that you will need to make is how you are going to orchestrate the streaming data processing in your pipeline.  There are several valid approaches.  First, you may choose to start the bronze_stream and let it complete (read and append all of the source data) before preceeding and starting up the silver_stream.  This approach has latency associated with it but it will allow your code to proceed in a linear fashion and process all the data by the end of your notebook execution.  Another potential approach is to start all the streams and have a "watch" method to determine when the pipeline has processed sufficient or all of the source data before stopping and displaying results.  Both of these approaches are valid and have different implications on how you will trigger your steams and how you will gate the execution of your pipeline.  Think through how you want to proceed and ask questions if you need guidance. The following references may be helpful:
- [Spark Structured Streaming Programming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Databricks Autoloader - Cloudfiles](https://docs.databricks.com/en/ingestion/auto-loader/index.html)

### Be sure that your project runs end to end when *Run all* is executued on this notebook! (15 Points out of 60)

In [0]:
! pip install pyspark

In [0]:
%run ./includes/includes

In [0]:
"""
Adding a widget to the notebook to control the clearing of a previous run.
or stopping the active streams using routines defined in the utilities notebook
"""
dbutils.widgets.removeAll()

dbutils.widgets.dropdown("clear_previous_run", "No", ["No","Yes"])
if (getArgument("clear_previous_run") == "Yes"):
    clear_previous_run()
    print("Cleared all previous data.")

dbutils.widgets.dropdown("stop_streams", "No", ["No","Yes"])
if (getArgument("stop_streams") == "Yes"):
    stop_all_streams()
    print("Stopped all active streams.")

from delta import *
dbutils.widgets.dropdown("optimize_tables", "No", ["No","Yes"])
if (getArgument("optimize_tables") == "Yes"):
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, BRONZE_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, SILVER_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, GOLD_DELTA).optimize().executeCompaction()
    print("Optimized all of the Delta Tables")

## 1.0 Import your libraries here...
- Are your shuffle partitions consistent with your cluster and your workload?
- Do you have the necessary libraries to perform the required operations in the pipeline/application?

In [0]:
# ENTER YOUR CODE HERE
from pyspark.sql.functions import input_file_name
from pyspark.sql.functions import current_timestamp
from pyspark.sql.functions import isnan
from pyspark.sql.functions import isnull

from pyspark.sql.functions import when
from pyspark.sql.functions import count
from pyspark.sql.functions import desc
from pyspark.sql.functions import col

from pyspark.sql.functions import to_date
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

from pyspark.sql.functions import length
import pyspark.sql.functions as F
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import expr
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType

from pyspark.sql.types import IntegerType
from pyspark.sql.types import TimestampType
from pyspark.sql.types import LongType
import matplotlib.pyplot as plt

import mlflow
from mlflow.pyfunc import spark_udf
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from pyspark.sql.functions import countDistinct

from pyspark.sql.window import Window
from pyspark.sql.functions import sum as spark_sum

start_time = time.time()


## 2.0 Use the utility functions to ...
- Read the source file directory listing
- Count the source files (how many are there?)
- print the contents of one of the files

In [0]:
# ENTER YOUR CODE HERE

# 1. Read the Source File Directory Listing
df = get_source_listing_df()
print(df)

# 2. Count the Source Files
file_count = len(df) - 1   # Subtract one to exclude the header row
print(f"Number of source files: {file_count}")

# 3. Print the Contents of One of the Files
if file_count > 0:
    filename = df.iloc[1]['File Name']  # Use the second row which should be the first actual data file if there's a header
    print(f"\nContents of {filename}:")
    file_contents = show_s3_file_contents(filename)
    print(file_contents.decode('utf-8'))  # Assuming the file content is in bytes
else:
    print("No files found in the source directory or only header is present.")

## 3.0 Transform the Raw Data to Bronze Data using a stream
- define the schema for the raw data
- setup a read stream using cloudfiles and the source data format
- setup a write stream using cloudfiles to append to the bronze delta table
- enforce schema
- allow a new schema to be merged into the bronze delta table
- Use the defined BRONZE_CHECKPOINT and BRONZE_DELTA paths defines in the includes
- name your raw to bronze stream as bronze_stream
- transform the raw data to the bronze data using the data definition at the top of the notebook

In [0]:
# JUNK
manual_read_file = spark.read.json(TWEET_SOURCE_PATH + "1.json")
read_data_schema =  manual_read_file.schema
print(read_data_schema)

In [0]:
#ENTER YOUR CODE HERE

#1 define the schema for raw data
rawDataSchema = StructType([
    StructField('date', StringType(), True),
    StructField('sentiment', StringType(), True),
    StructField('text', StringType(), True),
    StructField('user', StringType(), True),
])

bronzeSchema = StructType([
    StructField("date", StringType(), True),
    StructField("sentiment", StringType(), True),
    StructField("text", StringType(), True),
    StructField("user", StringType(), True),
    StructField("source_file", StringType(), True),
    StructField("processing_time_stamp", StringType(), True)
])

# dbutils.fs.rm(USER_DIR, True)
dbutils.fs.rm(USER_DIR + "/bronze.delta", True)
dbutils.fs.rm(USER_DIR + "/bronze.checkpoint", True)


In [0]:
%sql
DROP TABLE IF EXISTS bronze;
-- CREATE TABLE bronze (
-- date STRING,
-- sentiment STRING,
-- text STRING,
-- user STRING,
-- source_file_name STRING,
-- processing_time_stamp TIMESTAMP
-- )
-- USING delta
-- LOCATION '/tmp/labuser104917-3021490/bronze.delta'

In [0]:
#2 setup a read stream using cloudfiles
def streaming_bronze():
    print("Streaming read start")
    return (
        spark.readStream.format("cloudFiles")
        .option("cloudFiles.format", "json")
        .option("maxFilesPerTrigger", 1)
        .schema(rawDataSchema) #4 enforce schema
        .load(TWEET_SOURCE_PATH)
        .selectExpr("*","_metadata.file_name as source_file_name","_metadata.file_modification_time as processing_time_stamp")
    )

In [0]:
#3 setup a write stream using cloudfiles
def write_stream_to_delta_table(my_stream):
    print("Streaming write start")
    return (
        my_stream.writeStream
        .queryName("bronze_stream") #7 name your raw to bronze stream as bronze_stream
        .format("delta")
        .option("checkpointLocation", BRONZE_CHECKPOINT) #6 Use the defined BRONZE_DELTA 
        .option("mergeSchema", "true")#5 Merging Schemas
        .outputMode("append") 
        .start(BRONZE_DELTA) #6 Use the defined BRONZE_DELTA 
    )

In [0]:
#7 transform the raw data to the bronze data
my_stream = streaming_bronze()
activity_query = write_stream_to_delta_table(my_stream)
# activity_query.awaitTermination(80)
activity_query.awaitTermination()
activity_query.stop()

In [0]:
#Show 10 records
read_from_delta_df = spark.read.load(BRONZE_DELTA)
print(read_from_delta_df.show(10))

In [0]:
%sql
SET spark.sql.legacy.timeParserPolicy = LEGACY;

## 4.0 Bronze Data Exploratory Data Analysis
- How many tweets are captured in your Bronze Table?
- Are there any columns that contain Nan or Null values?  If so how many and what will you do in your silver transforms to address this?
- Count the number of tweets by each unique user handle and sort the data by descending count.
- How many tweets have at least one mention (@) how many tweet have no mentions (@)
- Plot a bar chart that shows the top 20 tweeters (users)


In [0]:
# ENTER YOUR CODE HERE
#1 How many tweets are captured
total_tweets_in_table = read_from_delta_df.count()
print("Total Number of rows = " , total_tweets_in_table)


In [0]:
#2 Are there any columns that contain Nan or Null values
dict_null = {col:read_from_delta_df.filter(read_from_delta_df[col].isNull()).count() for col in read_from_delta_df.columns}
print("Count of columns having nan/null column-wise")
print(dict_null)

#what will you do in your silver transforms to address this
print("In case of null values, an optimal choise would be to either discard the row, or assign a mean (in case of int values) or assign a default(in case of string values)")
print("Currently in this bronze data table, there are no null/NAN")


In [0]:
#3 Count the number of tweets by each unique user handle and sort the data by descending count.

streaming_data_count_read = spark.read.format("delta").load(BRONZE_DELTA)
count_query_df = streaming_data_count_read.groupBy("user").count().orderBy("count", ascending=False)
print(count_query_df.show(10))

In [0]:
#4 How many tweets have at least one/zero mention (@)

streaming_data_mention_read = spark.read.format("delta").load(BRONZE_DELTA)
mention_count = streaming_data_mention_read.filter(col("text").like("%@%")).count()
zero_mention_tweets = total_tweets_in_table - mention_count
print("Number of tweets with at least 1 mention: ", mention_count)

print("Number of tweets with 0 mentions: ", zero_mention_tweets)

In [0]:
#5 Plot a bar chart that shows the top 20 tweeters (users)
top_users_list = count_query_df.limit(20).toPandas()

plt.figure(figsize=(30, 10))
plt.bar(top_users_list['user'], top_users_list['count'])
plt.xlabel('List of Users')
plt.ylabel('Number of Tweets per User')
plt.title('Top 20 Tweeters')
plt.show()

## 5.0 Transform the Bronze Data to Silver Data using a stream
- setup a read stream on your bronze delta table
- setup a write stream to append to the silver delta table
- Use the defined SILVER_CHECKPOINT and SILVER_DELTA paths in the includes
- name your bronze to silver stream as silver_stream
- transform the bronze data to the silver data using the data definition at the top of the notebook

In [0]:
dbutils.fs.rm(USER_DIR + "/silver.delta", True)
dbutils.fs.rm(USER_DIR + "/silver.checkpoint", True)

In [0]:
%sql
DROP TABLE IF EXISTS silver;
-- CREATE OR REPLACE TABLE silver (
-- date STRING,
-- sentiment STRING,
-- text STRING,
-- user STRING,
-- source_file_name STRING,
-- processing_time_stamp TIMESTAMP,
-- conv_date TIMESTAMP
-- )
-- USING delta
-- LOCATION '/tmp/labuser104917-3021490/silver.delta'

In [0]:
#1 setup a read stream on your bronze delta table
def read_from_bronze():
    print("read from bronze")
    return (
        spark.readStream
        .format("delta")
        .load(BRONZE_DELTA)
    )

In [0]:
#2 setup a write stream to append to the silver delta table
def write_silver_stream(my_stream):
    print("write to Silver")
    return (
            my_stream.writeStream.format("delta")
                    .queryName("silver_stream") #4 Name your bronze to silver stream as silver_stream
                    .option("checkpointLocation", SILVER_CHECKPOINT) #3 Use the defined SILVER_CHECKPOINT and SILVER_DELTA
                    .outputMode("append")
                    .start(SILVER_DELTA)
             )

In [0]:
# ENTER YOUR CODE HERE
#5 Transform the bronze data to the silver data 
streaming_data_read_bronze = read_from_bronze()

streaming_data_read_bronze = streaming_data_read_bronze.withColumn("conv_date", to_date(streaming_data_read_bronze["date"], "EEE MMM dd HH:mm:ss zzz yyyy"))

streaming_data_read_bronze = streaming_data_read_bronze.withColumn("mentions", explode(split(col("text"), " "))).filter(col("mentions").startswith("@")).filter(col("mentions") != "@")

streaming_data_read_bronze = streaming_data_read_bronze.withColumn("cleaned_text", regexp_replace(col("text"), "@[a-zA-Z0-9_]+", ""))

selected_cols = ["conv_date", "mentions", "cleaned_text", "sentiment"]
streaming_data_read_bronze = streaming_data_read_bronze.select(selected_cols)

activity_query_silver = write_silver_stream(streaming_data_read_bronze)

# activity_query_silver.awaitTermination(80)
activity_query_silver.awaitTermination()
activity_query_silver.stop()

In [0]:
#Show 10 records
read_from_silver_df = spark.read.load(SILVER_DELTA)
print(read_from_silver_df.show(10))

## 6.0 Transform the Silver Data to Gold Data using a stream
- setup a read stream on your silver delta table
- setup a write stream to append to the gold delta table
- Use the defined GOLD_CHECKPOINT and GOLD_DELTA paths defines in the includes
- name your silver to gold stream as gold_stream
- transform the silver data to the gold data using the data definition at the top of the notebook
- Load the pretrained transformer sentiment classifier from the MODEL_NAME at the production level from the MLflow registry
- Use a spark UDF to parallelize the inference across your silver data

In [0]:
dbutils.fs.rm(USER_DIR + "/gold.delta", True)
dbutils.fs.rm(USER_DIR + "/gold.checkpoint", True)

In [0]:
# ENTER YOUR CODE HERE
#1 setup a read stream on your silver delta table
def read_from_silver():
    print("read from silver")
    return (
        spark.readStream
        .format("delta")
        .load(SILVER_DELTA)
)

#2 setup a write stream to append to the gold delta table
def write_to_gold(my_stream):
    print("write to Gold")
    return (
            my_stream.writeStream.format("delta")
                    .queryName("gold_stream") #4 Name your silver to gold stream as gold_stream
                    .option("checkpointLocation", GOLD_CHECKPOINT) #3 Use the defined GOLD_CHECKPOINT and GOLD_DELTA paths
                    .outputMode("append")
                    .start(GOLD_DELTA)
)

#6 Load the pretrained transformer sentiment classifier from the MODEL_NAME    
sentiment_function = spark_udf(spark, "models:/" + MODEL_NAME + "/production")

#5 Transform the silver data to the gold data
streaming_data_read_silver = read_from_silver()

streaming_data_read_silver = streaming_data_read_silver.withColumn("predicted", sentiment_function("cleaned_text"))

streaming_data_read_silver = streaming_data_read_silver.select("conv_date", "mentions", "cleaned_text", "sentiment", col("predicted.score").alias("predicted_score"), col("predicted.label").alias("predicted_sentiment"))

streaming_data_read_silver = streaming_data_read_silver.withColumn("sentiment_id", when(col("sentiment") == "positive", 1).otherwise(-1))

streaming_data_read_silver = streaming_data_read_silver.withColumn("predicted_sentiment_id", when(col("predicted_sentiment") == "POS", 1).otherwise(-1))

activity_query_gold = write_to_gold(streaming_data_read_silver)

# activity_query_gold.awaitTermination(160)
activity_query_gold.awaitTermination()
activity_query_gold.stop()

    

In [0]:
#Show 10 records
read_from_gold_df = spark.read.load(GOLD_DELTA)
print(read_from_gold_df.show(10))

## 7.0 Capture the accuracy metrics from the gold table in MLflow
Store the following in an MLflow experiment run:
- Store the precision, recall, and F1-score as MLflow metrics
- Store an image of the confusion matrix as an MLflow artifact
- Store the mdoel name and the MLflow version that was used as an MLflow parameters
- Store the version of the Delta Table (input-silver) as an MLflow parameter

In [0]:
# ENTER YOUR CODE HERE
#1 Store the precision, recall, and F1-score
predicted_data = read_from_gold_df.withColumn("predicted_sentiment_id", when(col("predicted_sentiment_id") == 0, -1).otherwise(col("predicted_sentiment_id")))
predicted_labels = predicted_data.select("predicted_sentiment_id").collect()

my_predictions = read_from_gold_df.select("predicted_sentiment_id").collect()
actual_values = read_from_gold_df.select("sentiment_id").collect()

model_recall = recall_score(actual_values, my_predictions)
model_f1 = f1_score(actual_values, my_predictions)
model_precision = precision_score(actual_values, my_predictions)

print("model_recall =", model_recall)
print("model_f1 =", model_f1)
print("model_precesion =", model_precision)

#2 Store an image of the confusion matrix
cm = confusion_matrix(actual_values, my_predictions)
my_confusion_matrix = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Negative", "Positive"])
my_confusion_matrix.plot()
dbutils.fs.mkdirs("dbfs:/CONF_MATRIX")
my_confusion_matrix.figure_.savefig("/dbfs/CONF_MATRIX/MATRIX_IMG.PNG")

#4 Store the version of the Delta Table (input-silver) 
silver_table_version = DeltaTable.forPath(spark, SILVER_DELTA).history().toPandas()["version"].iloc[-1]

with mlflow.start_run():
    mlflow.log_metric("model_recall", model_recall)
    mlflow.log_metric("model_f1", model_f1)
    mlflow.log_metric("model_precision", model_precision)
    mlflow.log_param("MODEL_NAME", MODEL_NAME) #3 Store the mdoel name and the MLflow version
    mlflow.log_param("mlflow_version", mlflow.version.VERSION)
    mlflow.log_param("silver_table_version", silver_table_version)

## 8.0 Application Data Processing and Visualization
- How many mentions are there in the gold data total?
- Count the number of neutral, positive and negative tweets for each mention in new columns
- Capture the total for each mention in a new column
- Sort the mention count totals in descending order
- Plot a bar chart of the top 20 mentions with positive sentiment (the people who are in favor)
- Plot a bar chart of the top 20 mentions with negative sentiment (the people who are the vilians)

You may want to use the "Loop Application" widget to control whether you repeateded display the latest plots while the data comes in from your streams before moving on to the next section and cleaning up your run.

*note: A mention is a specific twitter user that has been "mentioned" in a tweet with an @user reference.

In [0]:
# ENTER YOUR CODE HERE
#1 How many mentions are there in the gold data total?
total_mentions = read_from_gold_df.select(countDistinct("mentions")).collect()[0][0]
print("Total Mentions:", total_mentions)

#2 Count the number of neutral, positive and negative tweets 
grouped_df = read_from_gold_df.groupBy("mentions", "sentiment").count()
pivot_df = grouped_df.groupBy("mentions").pivot("sentiment").sum("count")
mention_counts_df = pivot_df.na.fill(0)
mention_counts_df.show()

#3 Capture the total for each mention in a new column
window_spec = Window.partitionBy("mentions")
mention_counts_df = mention_counts_df.withColumn("total", spark_sum(mention_counts_df["positive"] + mention_counts_df["negative"]).over(window_spec))
mention_counts_df.show()

#4 Sort the mention count totals in descending order
sorted_mention_counts_df = mention_counts_df.orderBy(mention_counts_df["total"].desc())
sorted_mention_counts_df.show()


In [0]:
#5 Plot a bar chart of the top 20 mentions with positive sentimen
top_mentions_positive = sorted_mention_counts_df.filter(sorted_mention_counts_df["positive"] > 0).limit(20)
mentions = top_mentions_positive.select("mentions").rdd.flatMap(lambda x: x).collect()
positive_counts = top_mentions_positive.select("positive").rdd.flatMap(lambda x: x).collect()
plt.figure(figsize=(12, 6))
plt.bar(mentions, positive_counts)
plt.xlabel("Mention")
plt.ylabel("Positive Count")
plt.title("Top 20 Mentions with Positive Sentiment")
plt.xticks(rotation=90)
plt.show()

#6 Plot a bar chart of the top 20 mentions with negative sentiment
top_mentions_negative = sorted_mention_counts_df.filter(sorted_mention_counts_df["negative"] > 0).limit(20)
mentions = top_mentions_negative.select("mentions").rdd.flatMap(lambda x: x).collect()
negative_counts = top_mentions_negative.select("negative").rdd.flatMap(lambda x: x).collect()
plt.figure(figsize=(12, 6))
plt.bar(mentions, negative_counts)
plt.xlabel("Mention")
plt.ylabel("Negative Count")
plt.title("Top 20 Mentions with Negative Sentiment")
plt.xticks(rotation=90)
plt.show()

## 9.0 Clean up and completion of your pipeline
- using the utilities what streams are running? If any.
- Stop all active streams
- print out the elapsed time of your notebook.

In [0]:
# ENTER YOUR CODE HERE

#1 using the utilities what streams are running? 
active_streams = spark.streams.active
for stream in active_streams:
    print(stream.name)

#2 Stop all active streams
for stream in spark.streams.active:
    print(f"Stopping stream {stream.name} (was active)")
    stream.stop()

elapsed_time = time.time() - start_time
print("Elapsed time:", elapsed_time)

## 10.0 How Optimized is your Spark Application (Grad Students Only)
Graduate students (registered for the DSCC-402 section of the course) are required to do this section.  This is a written analysis using the Spark UI (link to screen shots) that support your analysis of your pipelines execution and what is driving its performance.
Recall that Spark Optimization has 5 significant dimensions of considertation:
- Spill: write to executor disk due to lack of memory
- Skew: imbalance in partition size
- Shuffle: network io moving data between executors (wide transforms)
- Storage: inefficiency due to disk storage format (small files, location)
- Serialization: distribution of code segments across the cluster

Comment on each of the dimentions of performance and how your impelementation is or is not being affected.  Use specific information in the Spark UI to support your description.  

Note: you can take sreenshots of the Spark UI from your project runs in databricks and then link to those pictures by storing them as a publicly accessible file on your cloud drive (google, one drive, etc.)

References:
- [Spark UI Reference Reference](https://spark.apache.org/docs/latest/web-ui.html#web-ui)
- [Spark UI Simulator](https://www.databricks.training/spark-ui-simulator/index.html)

### ENTER YOUR MARKDOWN HERE